## Challenge Matplotlib

Martin Singla - Data Analyst

Coding challenge using Python Matplotlib framework , part of the Data Analytics Bootcamp program by Tecnológico de Monterrey

In [141]:
#loading dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#loading data
mouses = pd.read_csv("data/Mouse_metadata.csv")
results = pd.read_csv("data/Study_results.csv")

#Basic exploration of the df

#Mouses df
mouses.head()
len(data["Mouse ID"].unique()) #249 unique mouses
mouses["Mouse ID"].value_counts()[mouses["Mouse ID"].value_counts() > 1] #seems to be no repeated values
mouses["Drug Regimen"].value_counts() #10 drugs / 24-25 cases each
mouses["Sex"].value_counts() #50% -50% distribution by sex
mouses.describe() #get summary stats from AGE and WEIGHT variables, both seem relatively nrmally distributed

#Results df
results.head()
len(results["Mouse ID"].unique()) #also 249 unique mouses
len(results["Timepoint"].unique()) #10 unique timepoints
results["Timepoint"].unique() #which go 5 by 5...
results.iloc[:,2].describe() #tumor volume summary stats
results["Metastatic Sites"].unique() #what is this variabl??

#merging dfs
data = pd.merge(results, mouses, on= "Mouse ID", how= "left") 
data.head() #btw, shouldn´t WEIGHT be another experimental variable and not an ID descriptor?

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [142]:
#Data cleaning / removing duplicated cases

data[data.duplicated()] #one duplicated entry found (row id 137, mouse ID = g989)
data.loc[data["Mouse ID"] == "g989",:] #the entries for this rat are a mess, we have duplicated timepoints all over the place

data["Mouse ID"].unique() #examples: b128, f932, g107, a457, c819
data.loc[data["Mouse ID"] == "a457",:] #checking examples of timepoint series for each rat, there shouldn-t be duplicated values

a = data.groupby(["Mouse ID", "Timepoint"])["Timepoint"].count() 
a[a > 1] #effectively, Mouse ID g989 is the only case with duplicated timepoints.

data = data.loc[data["Mouse ID"] != "g989",:] #we don´t know what happened with the data entry of that mouse, let´s just drop it
len(data["Mouse ID"].unique()) #df with one less observation

248

In [143]:
#Quick summary stats table of TUMOR VOLUME by DRUG REGIMEN
data.groupby("Drug Regimen")["Tumor Volume (mm3)"].agg(["count", "mean", "median", "std", "var", "sem"])

,mean,median,std,var,sem
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,4.994774,24.947764,0.329346
Ceftamin,52.591172,51.776157,6.268188,39.290177,0.469821
Infubinol,52.884795,51.820584,6.567243,43.128684,0.492236
Ketapril,55.235638,53.698743,8.279709,68.553577,0.603860
Naftisol,54.331565,52.509285,8.134708,66.173479,0.596466
Placebo,54.033581,52.288934,7.821003,61.168083,0.581331
Propriva,52.320930,50.446266,6.622085,43.852013,0.544332
Ramicane,40.216745,40.673236,4.846308,23.486704,0.320955
Stelasyn,54.233149,52.431737,7.710419,59.450562,0.573111


In [160]:
#Other way of computing summary stats of TUMOR VOLUME by DRUG REGIMEN
a = data.groupby("Drug Regimen")["Tumor Volume (mm3)"]
pd.DataFrame({
    "Count": a.count(),
    "Mean": a.mean(), "Median": a.median(),
    "St.Dev.": a.std(),"Variance": a.var(), 
    "SEM": a.std()/np.sqrt(a.count())
})

,Count,Mean,Median,St.Dev.,Variance,SEM
Drug Regimen,,,,,,
Capomulin,230,40.675741,41.557809,4.994774,24.947764,0.329346
Ceftamin,178,52.591172,51.776157,6.268188,39.290177,0.469821
Infubinol,178,52.884795,51.820584,6.567243,43.128684,0.492236
Ketapril,188,55.235638,53.698743,8.279709,68.553577,0.603860
Naftisol,186,54.331565,52.509285,8.134708,66.173479,0.596466
Placebo,181,54.033581,52.288934,7.821003,61.168083,0.581331
Propriva,148,52.320930,50.446266,6.622085,43.852013,0.544332
Ramicane,228,40.216745,40.673236,4.846308,23.486704,0.320955
Stelasyn,181,54.233149,52.431737,7.710419,59.450562,0.573111
